In [1]:
#!pip3 install tensorflow
#!pip install tensorflow==1.14
#print('hello')

In [2]:
pwd

'/Users/bashirmohammed/Documents/GitHub/daphne/Bashirs_coderoom_testing/anl'

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import os
import pandas as pd
import numpy as np
#%matplotlib inline
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


/Users/bashirmohammed/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/bashirmohammed/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/bashirmohammed/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/bashirmohammed/anaconda3/lib/python3.7/site-pack

In [4]:
pwd

'/Users/bashirmohammed/Documents/GitHub/daphne/Bashirs_coderoom_testing/anl'

In [5]:
## Get working directory
PATH = os.getcwd()

In [6]:
## Path to save the embedding and checkpoints generated
LOG_DIR = PATH + '/project-tensorboard/log-1/'

In [7]:
## Load data
df = pd.read_csv("2018_1hourdata_labelled.csv")

In [8]:
del df['Time']

In [9]:
del df['label']

In [10]:
df.head()

,SACR_SUNN_in,SACR_SUNN_out,LOND_NEWY_in,LOND_NEWY_out,JGI_SACR_in,JGI_SACR_out,HOUS_PANTEX_in,HOUS_PANTEX_out,BOIS_PNWG_in,BOIS_PNWG_out,...,AMST_LOND_in,AMST_LOND_out,FNAL_STAR_in,FNAL_STAR_out,BOST_STAR_in,BOST_STAR_out,BOST_NEWY_in,BOST_NEWY_out,ATLA_WASH_in,ATLA_WASH_out
0,1.411093e+10,1.025131e+10,1.109243e+09,8.683116e+09,8.289831e+07,471952486.5,56650005.21,28102344.40,1.117868e+07,220096758.3,...,3.131858e+09,1.290804e+10,6.221437e+07,4.177833e+07,6.127005e+09,1.505475e+10,1.116761e+10,2.391813e+09,3.005335e+09,5.993152e+09
1,1.345362e+10,9.191558e+09,1.242256e+09,5.181239e+09,4.375739e+04,470169221.0,49307351.26,20240358.57,7.070847e+06,224528257.8,...,4.157574e+09,7.468747e+09,1.943452e+08,3.058273e+08,4.952796e+09,1.096238e+10,7.879534e+09,2.225971e+09,3.043124e+09,4.402875e+09
2,1.216888e+10,7.793842e+09,2.006322e+09,1.224286e+10,4.315725e+04,254319077.6,56843652.59,13076249.56,5.132826e+07,229200120.9,...,6.164468e+09,1.451626e+10,2.368597e+08,2.297360e+08,7.193835e+09,1.840171e+10,1.573402e+10,2.598615e+09,3.254450e+09,3.451973e+09
3,1.123120e+10,7.097238e+09,1.073181e+09,4.717642e+09,8.277154e+07,167067135.5,46645240.59,15344128.69,5.217827e+06,212289872.7,...,3.856222e+09,7.139046e+09,2.644591e+07,8.776042e+07,4.389148e+09,7.153226e+09,7.690503e+09,1.992397e+09,3.717638e+09,5.663406e+09
4,1.078085e+10,8.048294e+09,8.645069e+08,1.987249e+09,3.343548e+04,113702282.5,30966947.37,23023533.87,7.495363e+06,213758038.0,...,2.483093e+09,4.691328e+09,3.782775e+07,7.183373e+07,2.982568e+09,4.568829e+09,4.828736e+09,2.083069e+09,3.753083e+09,4.915730e+09


In [11]:
# #df = StandardScaler().fit_transform(df)


# x = df.values #returns a numpy array
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# df = pd.DataFrame(x_scaled)

In [12]:
df.shape

(8760, 96)

In [13]:
metadata = os.path.join(LOG_DIR, 'df.tsv')

In [14]:
# Generating PCA and 
pca = PCA(n_components=96,
         random_state = 123,
         svd_solver = 'auto'
         )

df_pca = pd.DataFrame(pca.fit_transform(df))
df_pca = df_pca.values

In [15]:
## TensorFlow Variable from data
tf_data = tf.Variable(df_pca)

In [16]:
## Running TensorFlow Session
with tf.Session() as sess:
    saver = tf.train.Saver([tf_data])
    sess.run(tf_data.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'tf_data.ckpt'))
    config = projector.ProjectorConfig()
    

In [17]:
# One can add multiple embeddings.
embedding = config.embeddings.add()
embedding.tensor_name = tf_data.name

In [18]:
# Link this tensor to its metadata(Labels) file
embedding.metadata_path = metadata

In [19]:
# Saves a config file that TensorBoard will read during startup.
projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

In [20]:
print(config)

embeddings {
  tensor_name: "Variable:0"
  metadata_path: "/Users/bashirmohammed/Documents/GitHub/daphne/Bashirs_coderoom_testing/anl/project-tensorboard/log-1/df.tsv"
}



In [21]:
print('hello')

hello


In [ ]:
# import pkg_resources

# for entry_point in pkg_resources.iter_entry_points('tensorboard_plugins'):
#     print(entry_point.dist)

In [ ]:
#!sudo pip list --format=freeze | grep tensorboard | xargs pip uninstall -y

In [ ]:
#tensorboard --logdir /Users/bashirmohammed/Dropbox/CodeRoom/Python/python3_jose/project-tensorboard/log-1/ --port=6006